In [1]:
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib as mpl
#mpl.rcParams['axes.linewidth'] = 2
#import pandas as pd
#import math 
import os
import pickle
#import scipy.ndimage as ndimage
#from matplotlib.ticker import AutoMinorLocator
import json
import sys
from pdf2image import convert_from_path
#project_folder = "C:\\Users\\giogu\\Documents\\Acustica\\python_routines" 
#sys.path.append(project_folder)

sys.path.append("../../")

from pyAcousticsAnalysis.instruments.LD_831.read_data import read_LD_831_data
from pyAcousticsAnalysis.instruments.commons import read_preprocessed_data#, get_masks
from pyAcousticsAnalysis.utils.helper_functions import get_mean_levels, get_min_levels, generate_freq_third_octaves, logarithmic_average
#from pyEnvAcoustics.plotting.plot_functions import plot_mean_Leq, get_freq_x_ticks
from pyAcousticsAnalysis.plotting.plot_SPL import plot_SPL
from pyAcousticsAnalysis.plotting.plot_time_history import plot_time_history
from pyAcousticsAnalysis.plotting.plot_min_SPL import plot_min_SPL
from pyAcousticsAnalysis.plotting.helper_functions import calculate_quantiles, calculate_global_quantities
from pyAcousticsAnalysis.plotting.latex_functions import create_pdfs, merge_pdfs

%load_ext autoreload
%autoreload 2

### Folders

In [3]:
base_folder = "../../formificio_Pistoia"

In [4]:
foto_folder =  os.path.join(base_folder, "foto").replace("\\", "/")
#logo_folder =  os.path.join(base_folder, "logo").replace("\\", "/")
logo_folder =  os.path.join(base_folder, "../logos").replace("\\", "/")
reports_folder =  os.path.join(base_folder, "reports").replace("\\", "/")
preprocessed_folder = os.path.join(reports_folder, "preprocessed").replace("\\", "/")
data_folder = os.path.join(reports_folder, "data").replace("\\", "/")

plots_folder = os.path.join(reports_folder, "plots").replace("\\", "/")
os.makedirs(plots_folder, exist_ok=True) 

In [5]:
with open(os.path.join(base_folder, "metadata.json"), 'r') as f:
    metadata = json.load(f)

In [6]:
#filenames = sorted([x for x in os.listdir(preprocessed_folder) if x.endswith(".xlsx")])
filenames = [x['filename'] for x in metadata]

out_list = []

for filename in filenames[:]:
    print(filename)
    nome_misura = filename.split("-")[-1].replace(".xlsx", ".s")
    file_path = os.path.join(preprocessed_folder, filename)
    
#    df = read_LD_831_data(file_path)
    df = read_preprocessed_data(file_path)
        
    #plot
    plot_SPL(nome_misura, filename, df, plots_folder, show = False)
    plot_time_history(nome_misura, filename, df, plots_folder, find_impulses = False, show = False)    
    plot_min_SPL(nome_misura, filename, df, plots_folder, show = False)
    quantiles = calculate_quantiles(df)
    global_quantities = calculate_global_quantities(df)
#    print(quantiles)
    
    out_dict = {"filename": filename, 
                "fields": 
                    {
#                    "-logo-": logo_folder + "/" + os.listdir(logo_folder)[0][:-4], 
                    "-logo-": logo_folder + "/logo_nier",                     
                    "-nome!misura-": nome_misura.replace("_", "\_"), 
                   "-data!ora-": str(df.loc[0, 'Data']),
                    "-SPL!plot-": plots_folder + "/" + filename[:-5] + "_SPL",                       
#                    "-time!history!plot-": "../plots/" + filename[:-5] + "_time_history",
                    "-time!history!plot-":  plots_folder + "/" +filename[:-5] + "_time_history",
                    "-SPL!min!plot-":  plots_folder + "/" + filename[:-5] + "_SPL_min"
                    }
               }    
    
    #add quantiles
    out_dict['fields'].update(quantiles)

    #add global_quantities
    out_dict['fields'].update(global_quantities)
    
    #add metadata    
    metadata_misura = [x for x in metadata if x['filename'] == out_dict['filename']][0]
    out_dict['fields'].update({k: v for k, v in metadata_misura.items() if k != "filename"})

    #update foto path
    if out_dict['fields']['-foto-'] != "":
        out_dict['fields']['-foto-'] = foto_folder + "/" + out_dict['fields']['-foto-']
    else:
#        print("here")
        out_dict['fields']['-foto-'] = logo_folder + "/" + "blank_photo" 
        
    #append results
    out_list.append(out_dict)

#    print(metadata_misura)

#    print(df[df['masked'] == True].shape)
    
with open(os.path.join(data_folder, "latex_data.json"), 'w') as f:
    json.dump(out_list, f)

print("end")

<>:27: SyntaxWarning: invalid escape sequence '\_'
<>:27: SyntaxWarning: invalid escape sequence '\_'
/tmp/ipykernel_36126/1862352144.py:27: SyntaxWarning: invalid escape sequence '\_'
  "-nome!misura-": nome_misura.replace("_", "\_"),


831_0003315-20240621 143901-447TH_M.046.xlsx
831_0003315-20240621 145009-447TH_M.047.xlsx
831_0003315-20240621 150538-447TH_M.048.xlsx
831_0003315-20240621 151626-447TH_M.049.xlsx
831_0003315-20240621 152737-447TH_M.050.xlsx
831_0003315-20240621 153806-447TH_M.051.xlsx
831_0003315-20240621 154949-447TH_M.052.xlsx
831_0003315-20240621 164812-447TH_M.055.xlsx
831_0003315-20240621 160722-447TH_M.053.xlsx
831_0003315-20240621 162244-447TH_M.054.xlsx
end


### Create latex and pdfs

In [7]:
templates_folder = os.path.join("../" "assets", "latex_templates", "ambientale_447")

In [8]:
create_pdfs(templates_folder, reports_folder)

creating latex for: 831_0003315-20240621 143901-447TH_M.046.xlsx
creating latex for: 831_0003315-20240621 145009-447TH_M.047.xlsx
creating latex for: 831_0003315-20240621 150538-447TH_M.048.xlsx
creating latex for: 831_0003315-20240621 151626-447TH_M.049.xlsx
creating latex for: 831_0003315-20240621 152737-447TH_M.050.xlsx
creating latex for: 831_0003315-20240621 153806-447TH_M.051.xlsx
creating latex for: 831_0003315-20240621 154949-447TH_M.052.xlsx
creating latex for: 831_0003315-20240621 164812-447TH_M.055.xlsx
creating latex for: 831_0003315-20240621 160722-447TH_M.053.xlsx
creating latex for: 831_0003315-20240621 162244-447TH_M.054.xlsx
- all latex files created


In [9]:
merge_pdfs(reports_folder, out_filename = "misure.pdf")

merging 831_0003315-20240621 143901-447TH_M.046.pdf
merging 831_0003315-20240621 145009-447TH_M.047.pdf
merging 831_0003315-20240621 150538-447TH_M.048.pdf
merging 831_0003315-20240621 151626-447TH_M.049.pdf
merging 831_0003315-20240621 152737-447TH_M.050.pdf
merging 831_0003315-20240621 153806-447TH_M.051.pdf
merging 831_0003315-20240621 154949-447TH_M.052.pdf
merging 831_0003315-20240621 164812-447TH_M.055.pdf
merging 831_0003315-20240621 160722-447TH_M.053.pdf
merging 831_0003315-20240621 162244-447TH_M.054.pdf
- all pdf merged


### Crop pdfs

In [10]:
images = convert_from_path(os.path.join(reports_folder, "misure.pdf"))

In [11]:
crop_folder = os.path.join(reports_folder, "pdf_crops")
os.makedirs(crop_folder, exist_ok=True) 

for ind, image in enumerate(images[0:]):

    width, height = image.size

    width_crop = 80
    height_crop = 10
    
    left = width_crop
    top = height_crop
    right = width - width_crop
    bottom = height - height_crop

    cropped_image = image.crop((left, top, right, bottom)) 
    cropped_image.save(os.path.join(crop_folder, str(ind + 1) +'.jpg'), 'JPEG')

print("end")

end


In [ ]:
#images[0]

In [ ]:
pdfs_folder = os.path.join(reports_folder, "pdfs")

In [ ]:
filenames = os.listdir(pdfs_folder)

In [ ]:
filenames = [os.path.join(pdfs_folder, x) for x in os.listdir(pdfs_folder)]

In [ ]:
filenames

In [ ]:
reports_folder.sort(key=lambda x: os.path.getmtime(x))

In [ ]:
filenames.sort(key=os.path.getctime)

In [ ]:
[x.split("/")[-1] for x in filenames]

In [ ]:
pdfs_folder = os.path.join(reports_folder, "pdfs")

In [ ]:
from PyPDF2 import PdfWriter, PdfReader

In [ ]:
filename_pdfs = os.listdir(pdfs_folder)

In [ ]:
for filename_pdf in filename_pdfs[0:1]:
    print(filename_pdf)

    with open(os.path.join(pdfs_folder, filename_pdf), "rb") as in_f:
        input1 = PdfReader(in_f)
        output = PdfWriter()
        
        numPages = len(input1.pages)
        print("document has %s pages." % numPages)

        for i in range(numPages):
            page = input1.pages[i]
            print(page.mediabox.getUpperRight_x(), page.mediaBox.getUpperRight_y())
            page.trimBox.lowerLeft = (25, 25)
            page.trimBox.upperRight = (225, 225)
            page.cropBox.lowerLeft = (50, 50)
            page.cropBox.upperRight = (200, 200)
            output.addPage(page)

In [ ]:



    with open("out.pdf", "wb") as out_f:
        output.write(out_f)